In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os

# --- 1. SETUP: Find a sample file ---
# We use a specific noisy file if you have it, or just the first one found
target_file = '../data/raw/1-100032-A-0.wav' # Start with this dog bark
if not os.path.exists(target_file):
    # Fallback: Just grab the first file in the folder
    import glob
    files = glob.glob('../data/raw/*.wav')
    target_file = files[0]

print(f"Analyzing: {os.path.basename(target_file)}")

# --- 2. LOAD & PREPROCESS ---
# Load audio at 16kHz (Standard for your project)
y, sr = librosa.load(target_file, sr=16000)

# --- 3. CREATE MEL SPECTROGRAM (The "Before" Image) ---
# n_mels=128 (Height), hop_length=512 (Stride)
# power=1 (Magnitude) is better for PCEN than power=2
S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=512, n_mels=128, power=1)

# Convert to dB for visualization (The "Standard" way humans look at audio)
S_db = librosa.power_to_db(S, ref=np.max)

# --- 4. CREATE PCEN SPECTROGRAM (The "After" Image) ---
# We scale S by 2**31 to match the integer assumption of the paper
# Parameters: gain=0.8 (removes most wind), bias=10 (lifts silence), time_constant=0.060 (fast tracking)
pcen_S = librosa.pcen(S * (2**31), 
                      sr=sr, 
                      gain=0.8, 
                      bias=10, 
                      power=0.25, 
                      time_constant=0.060, 
                      eps=1e-6)

# --- 5. VISUALIZATION (The "Color Filling") ---
plt.figure(figsize=(12, 10))

# Plot 1: Standard Mel-Spectrogram (dB)
plt.subplot(2, 1, 1)
# specshow automatically maps the matrix values to colors
img1 = librosa.display.specshow(S_db, x_axis='time', y_axis='mel', sr=sr, fmax=8000, cmap='magma')
plt.colorbar(img1, format='%+2.0f dB')
plt.title('Standard Mel-Spectrogram (Log Scale)\nNotice the vertical "smear" of background noise')

# Plot 2: PCEN Spectrogram
plt.subplot(2, 1, 2)
# We use a different colormap ('viridis') to distinguish it, or stick to 'magma'
img2 = librosa.display.specshow(pcen_S, x_axis='time', y_axis='mel', sr=sr, fmax=8000, cmap='magma')
plt.colorbar(img2)
plt.title('PCEN Enhanced Spectrogram\nNotice the background is darker, and "events" are sharper')

plt.tight_layout()
plt.show()